# Runnable

`Runnable`이란 invoked, batched, streamed, transformed and composed 될 수 있는 일의 단위이다.
- invoke/ainvoke: 하나의 input을 output으로 변경시켜주는 메서드
- batch/abatch: 여러 개의 input을 output으로 변경시켜주는 메서드
- stream/astream: 결과물을 stream 해준다.
- astream_log: 출력 및 입력에서 선택된 중간 결과를 Stream 해준다.

`Runnable`을 구성하는 주요 primitive(구성 요소)는 `RunnableSequence` and `RunnableParallel`이다.
- `RunnableSequence`는 한 `Runnable`의 결과물을 다음 과정의 input으로 사용함으로써 `Runnable`들을 순차적으로 invoke한다.
    - | 연산자를 사용하거나 Runnable의 list를 통해서 구성한다.
- `RunnableParallel`은 동일한 input을 여러 `Runnable`들에 전달함으로써 `Runnable`들을 동시에 invoke한다.
    - dict 타입을 활용해서 구성한다.

# 실습

## 1. RunnableSequence

In [3]:
from langchain_core.runnables import RunnableLambda

# | 연산자를 통해 RunnableSequence를 구성하였다
sequence = RunnableLambda(lambda x : x + 1) | RunnableLambda(lambda x : x * 2)
sequence.invoke(1)

4

1이 들어가서 `RunnableLambda`에 의해 1+1 연산이 먼저 이루어지고 

그 결과인 2가 두번째 `RunnableLambda`의 입력값으로 들어가서 2 * 2 연산이 이루어진다.

최종 결과는 4

In [2]:
sequence.batch([1,2,3])

[4, 6, 8]

앞선 과정이 병렬적으로 동시에 일어난다.

## 2. Runnable Parallel

In [6]:
# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
sequence.invoke(1)

{'mul_2': 4, 'mul_5': 10}

위 코드는 우선 첫번째 `RunnableLambda`가 수행된 이후 2가지 `RunnableLambda`가 병렬적으로 수행된다.

이런 특성을 활용하면 서로 다른 2가지 Retriever를 동시에 적용하여 문서를 검색할 수 있다.

가령 lexical search(키워드 기반)와 semantic search(유사도 기반)를 따로 수행하고 검색한 문서를 기반으로 ReRanking을 수행해서 LLM의 최종 답변을 얻을 수 있다.

## 3. Runnable Method 사용하기

모든 Runnable은 동작을 수정(실패시 재시도, configurable하게 만들기)하는 데 사용할 수 있는 추가 메서드가 있다.

In [8]:
from langchain_core.runnables import RunnableLambda

import random
def add_one(x: int) -> int:
    return x + 1

def buggy_double(y: int) -> int:
    # 70%의 확률로 fail하는 buggy code
    if random.random() > 0.3 :
        print("This code failed, will probably be retried")
        raise ValueError("Triggerd Buggy Code!")
    return y * 2

sequence = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry(    # 실패시 재시도하는 메서드
        stop_after_attempt=10,
        wait_exponential_jitter=False
    )
)

print(sequence.input_schema.schema())
print(sequence.output_schema.schema())
print(sequence.invoke(2))

{'title': 'add_one_input', 'type': 'integer'}
{'title': 'buggy_double_output', 'type': 'integer'}
This code failed, will probably be retried
This code failed, will probably be retried
This code failed, will probably be retried
6


출처 : [링크](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html)